In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./modules")

import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

from Potential import Potential
from Channel import Channel
from Scattering import Scattering
from ScatteringExp import ScatteringExp
from Grid import Grid
import RseSolver
from scipy.special import factorial2, spherical_jn
from plots.rc_params import *

In [2]:
potentialArgs = {"label": "minnesota", "kwargs": {"potId": 213}}
l = 0
channel = Channel(S=0, L=l, LL=l, J=l, channel=0)
potential = Potential(channel, **potentialArgs)
E_MeV = 50
scattExp = ScatteringExp(E_MeV=E_MeV, potential=potential)
rmatch = 12
grid = Grid(1e-24, rmatch, numIntervals=1, numPointsPerInterval=1000,
            type="linear", test=False)  # needs equidistant grids as of now
trainingLecList, testingLecList = Potential.getSampleLecs(potentialArgs["label"])

emulator_args = dict(scattExp=scattExp, 
                          grid=grid, 
                          free_lecs={"V1": (-400, 0)},
                          num_snapshots_init=80, 
                          num_snapshots_max=100,
                          approach="pod", 
                          pod_rcond=1e-12, 
                          init_snapshot_lecs=None,
                          greedy_max_iter=5, 
                          mode="linear")

In [3]:
lecs = np.array([[1,200,-91.85], [1,200,-91.85]])

In [4]:
from NumerovEmulator import MatrixNumerovROM_ab, MatrixNumerovROM, AllAtOnceNumerovROM
# Test config for no greedy
emulator_T = AllAtOnceNumerovROM(**emulator_args)
emulator_ab = MatrixNumerovROM_ab(**emulator_args)
emulator = MatrixNumerovROM(**emulator_args)

using 8 out of 80 POD modes in total: compression rate is 90.0 %
POD[ Y ]: compression rate is 33.3 %; dim: (1000, 18)
using 8 out of 80 POD modes in total: compression rate is 90.0 %
POD[ Y ]: compression rate is 29.6 %; dim: (1000, 19)
testing
using 8 out of 80 POD modes in total: compression rate is 90.0 %
POD[ Y ]: compression rate is 29.6 %; dim: (998, 19)


In [5]:
# %timeit emulator_T.simulate(lecs) 
# %timeit emulator_T.emulate(lecs, mode="lspg", lspg_solver="qr") 

In [6]:
def get_Kmat(S):
    return (1-S)/(1+S)*1j
print(get_Kmat(emulator_T.simulate(lecs, which="S")))
print(emulator_T.simulate(lecs, which="S"))
print(emulator_T.simulate(lecs, which="K"))
print(emulator_T.simulate(lecs, which="T"))
print(emulator_ab.simulate(lecs, which="K"))
print(emulator.simulate(lecs, which="S"))

[0.30244898+2.36508369e-10j 0.30244898+2.36508369e-10j]
[0.83238213+0.55420211j 0.83238213+0.55420211j]
[0.30244898+2.36508369e-10j 0.30244898+2.36508369e-10j]
[0.27710106+0.08380893j 0.27710106+0.08380893j]
[0.30244898 0.30244898]
[0.83238213+0.55420211j 0.83238213+0.55420211j]


In [7]:
np.max(np.abs(emulator.simulate(lecs, which="all")- emulator_ab.simulate(lecs, which="all")[:-2]))

1.3655743202889425e-14

In [8]:
def get_Kmat(S):
    return (1-S)/(1+S)*1j
print(get_Kmat(emulator_T.emulate(lecs, which="S")))
print(emulator_T.emulate(lecs, which="K"))
print(emulator_T.emulate(lecs, which="T"))
print(emulator_ab.emulate(lecs, which="K"))
print(emulator.emulate(lecs, which="K"))

[0.30244898+2.36135042e-10j 0.30244898+2.36135042e-10j]
[0.30244898+2.36135056e-10j 0.30244898+2.36135056e-10j]
[0.27710106+0.08380893j 0.27710106+0.08380893j]
[0.30244898 0.30244898]
diff [[ 0.00000000e+00  0.00000000e+00]
 [-3.88578059e-16 -3.88578059e-16]]
[0.30244898 0.30244898]


In [9]:
T = 0.13855053+0.04190447j
T / (1 + 1j * T)

(0.14784348444000545+0.022357558513805498j)

In [10]:
emulator_ab.simulate(lecs, which="all")[-2:]

array([[0.38987919, 0.38987919],
       [0.42036755, 0.42036755]])

In [11]:
# print(emulator_T.emulate(lecs, which="all")[-1])
print(emulator_ab.emulate(lecs, which="K"))
# print(emulator.emulate(lecs, which="T"))

[0.30244898 0.30244898]


In [12]:
np.max(np.abs(emulator.simulate(lecs) - emulator_ab.simulate(lecs)[:-2,:]))

1.3655743202889425e-14

In [13]:
aa = emulator_T.emulate_Tmatrix(lecs, estimate_norm_residual=True)
1 + 2j * aa

AttributeError: 'AllAtOnceNumerovROM' object has no attribute 'emulate_Tmatrix'

In [ ]:
Smat = emulator_T.emulate_Smatrix(lecs, estimate_norm_residual=True)
# np.abs(emulator_T.emulate_Smatrix(lecs, estimate_norm_residual=True))
np.log(Smat)/(2j)


array([-0.50738448+0.43454838j])

In [ ]:
np.max(np.abs(emulator_T.simulate(lecs)[:-1,:] - emulator_ab.simulate(lecs)[:-2,:]))

1.556367162194613

In [ ]:
aaa = np.array([1,2,3])
a = aaa[0]
a = 333
aaa

array([1, 2, 3])